# cli

> This module contains implementation for all the command line apps

In [ ]:
#| default_exp cli

In [ ]:
#|hide
from nbdev.showdoc import *
from dotenv import *

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
#|export
from fastcore.script import call_parse
import pathlib
import os
from fastlite import *
from rich.console import Console
from starlette.routing import Mount, Route
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import time

In [ ]:
#| export
console = Console()

In [ ]:
#| export
#| notest
@call_parse
def chelp():
    "Show help for all console scripts"
    from fastcore.xtras import console_help
    console_help('memexplatform_obsidian')

In [ ]:
# #| notest
# chelp()

In [ ]:
#| export

@call_parse
def sync_vault(reindex=False):
    "Initialize the databasee"
    from memexplatform_obsidian.datastore import DBStore
    from memexplatform_obsidian.commons import config
    store = DBStore(config)
    store.initdb()
    store.sync_vault(reindex=reindex)

In [ ]:
#| export
class VaultEventHandler(FileSystemEventHandler):
    def __init__(self, debounce: float = 2.0):
        super().__init__()
        self.debounce = debounce
        self.last_sync = 0

    def on_any_event(self, event):
        now = time.time()
        if now - self.last_sync > self.debounce:
            print(f"🔄 Detected changes in vault, syncing...")
            sync_vault()
            self.last_sync = now


@call_parse
def watch_vault(vault_path: str = None, debounce: float = 2.0):
    """
    Watch a vault directory for changes and run sync_vault() automatically.
    
    - vault_path: Path to Obsidian vault (default from config.vault_path if not provided).
    - debounce: Seconds to wait before re-triggering sync after a change.
    """

    from memexplatform_obsidian.datastore import DBStore
    from memexplatform_obsidian.commons import config
    if vault_path is None:
        vault_path = config.OBSIDIAN_VAULT

    vault_path = pathlib.Path(vault_path).expanduser().resolve()
    if not vault_path.exists():
        raise FileNotFoundError(f"Vault path does not exist: {vault_path}")

    event_handler = VaultEventHandler(debounce=debounce)
    observer = Observer()
    observer.schedule(event_handler, str(vault_path), recursive=True)

    print(f"👀 Watching vault at {vault_path}")
    observer.start()
    try:
        while True:
            time.sleep(1)
    except KeyboardInterrupt:
        print("🛑 Stopping vault watcher...")
        observer.stop()
    observer.join()

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()